In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import nltk
import re
import os
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import glob
from collections import Counter
from imblearn.over_sampling import SMOTE
# Any results you write to the current directory are saved as output.

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
df=pd.read_csv("../input/news-summary/news_summary_more.csv")

In [3]:
df.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [4]:
df1=df.sample(30000)

In [5]:
text=df1.text

In [6]:
headlines=df1.headlines

In [7]:
text.head()

17376    A 48-year-old vegetable vendor named Dadarao B...
21627    The Turkish lira has become more volatile than...
42000    The Indian Railways on Monday revealed that it...
4345     Following a phone call with Nelson Mandela in ...
91536    Former India hockey captain and Rajya Sabha MP...
Name: text, dtype: object

In [8]:
def preprocess(text):
    text=text.lower()
    text=text.strip()
    #text=text.replace("'s","",text)
    text=re.sub(r'[^\w\d]'," ",text)
    text=re.sub(r'\d+','num',text)
    text=re.sub(r"¹","",text)
    text=re.sub(r"\s\w\s"," ",text)
    text=re.sub(r"\s{2,}"," ",text)
    text=text.strip()
    return text

In [9]:
preprocess(text.iloc[3])

'following phone call with nelson mandela in june num former uk pm margaret thatcher had told her then foreign affairs adviser that the ex south african president had closed mind according to newly released secret files the uk then ambassador to south africa sir robin renwick had said mandela wasn as intelligent as ex zimbabwe president robert mugabe the files further revealed'

In [10]:
processtext=text.apply(lambda x:preprocess(x))

In [11]:
processheadline=headlines.apply(lambda x:preprocess(x))

In [12]:
processheadline.iloc[0]

'man fills num potholes in mumbai in num yrs after son death'

In [13]:
processtext.iloc[0]

'a num year old vegetable vendor named dadarao bilhore has filled in almost num potholes across mumbai over the past three years after his son died in road accident bilhore num year old son was travelling pillion on motorbike which hit pothole in july num also don want anyone else to lose loved one like we have bilhore said'

In [14]:
textwordcount=Counter()
for i in range(len(processtext)):
    textwordcount.update(processtext.iloc[i].split())

In [15]:
headlinewordcount=Counter()
for i in range(len(processheadline)):
    headlinewordcount.update(processheadline.iloc[i].split())

In [16]:
len([x for x,i in textwordcount.items() if i>2])

23293

In [17]:
len([x for x,i in headlinewordcount.items() if i>1])

12816

In [18]:
textokenizer=Tokenizer(num_words=24000,oov_token='<unk>')


In [19]:
textokenizer.fit_on_texts(processtext)
texttoken=textokenizer.texts_to_sequences(processtext)

In [20]:
textwordind=textokenizer.word_index
textwordind=dict([(i,j) for i,j in textwordind.items() if j<=25000])

In [21]:
textwordind['<pad>']=0

In [22]:
headtokenizer=Tokenizer(num_words=12500,oov_token='<unk>')
headtokenizer.fit_on_texts(processheadline)

In [23]:
headwordind=headtokenizer.word_index
headwordind=dict([(i,j) for i,j in headwordind.items() if j<=13500])

In [24]:
headwordind['<pad>']=0

In [25]:
headwordind['<start>']=len(headwordind)
headwordind['<end>']=len(headwordind)

In [26]:
headlinetoken=headtokenizer.texts_to_sequences(processheadline)

In [27]:
headlinetokenin=[[headwordind['<start>']]+i for i in headlinetoken]

In [28]:
headlinetokenout=[i+[headwordind['<end>']] for i in headlinetoken]

In [29]:
headlinetokenin[:4]

[[13501, 18, 8002, 2, 4907, 4, 62, 4, 2, 125, 10, 117, 88],
 [13501, 2849, 8003, 214, 131, 1, 126, 480],
 [13501, 570, 71, 13, 2, 78, 4499, 5, 2, 63, 547, 1],
 [13501, 50, 91, 22, 9742, 1236, 8004, 4908, 279, 2479, 2711, 6071]]

In [30]:
headlinetokenout[:4]

[[18, 8002, 2, 4907, 4, 62, 4, 2, 125, 10, 117, 88, 13502],
 [2849, 8003, 214, 131, 1, 126, 480, 13502],
 [570, 71, 13, 2, 78, 4499, 5, 2, 63, 547, 1, 13502],
 [50, 91, 22, 9742, 1236, 8004, 4908, 279, 2479, 2711, 6071, 13502]]

In [31]:
texttokenpad=pad_sequences(texttoken,padding='post')

In [32]:
headlineinpad=pad_sequences(headlinetokenin,padding='post')
headlineoutpad=pad_sequences(headlinetokenout,padding='post')

In [33]:
texttokenpad.shape

(30000, 74)

In [34]:
headlineinpad.shape

(30000, 17)

In [35]:
x=tf.placeholder(shape=[None,texttokenpad.shape[1]],dtype=tf.int32)
yinp=tf.placeholder(shape=[None,headlineinpad.shape[1]],dtype=tf.int32)
yout=tf.placeholder(shape=[None,headlineoutpad.shape[1]],dtype=tf.int32)

In [36]:
maxtextlen=texttokenpad.shape[1]
maxsumlen=headlineinpad.shape[1]

In [37]:
maxtextlen

74

In [38]:
def positional_embedding(pos, model_size):
    PE = np.zeros((1, model_size))
    for i in range(model_size):
        if i % 2 == 0:
            PE[:,i] = np.sin(pos / 10000 ** (i / model_size))
        else:
            PE[:,i] = np.cos(pos / 10000 ** ((i - 1) / model_size))
    return PE

# max_length = max(len(data_en[0]), len(data_fr_in[0]))
MODEL_SIZE = 128

pes = []
for i in range(maxtextlen):
    pes.append(positional_embedding(i, MODEL_SIZE))

pes = np.concatenate(pes, axis=0)
pes = tf.constant(pes, dtype=tf.float32)

In [39]:
modelsize=128
h=2
numlayers=3


In [40]:
def multiheadattention(modelsize,h):
    querysize=modelsize//h  # querysize and valuesize are to be same 
    valuesize=modelsize//h
    keysize=modelsize//h
    wq=[tf.keras.layers.Dense(querysize,activation='relu') for _ in range(h)]
    wk=[tf.keras.layers.Dense(querysize,activation='relu') for _ in range(h)]
    wv=[tf.keras.layers.Dense(querysize,activation='relu') for _ in range(h)]
    wo=tf.keras.layers.Dense(modelsize)
    return wq,wk,wv,wo,keysize



In [41]:
def call(query,value,keysize,h,wq,wk,wv,wo):
    # query len and keylen can be different size 
    # they are same only in encoder but for decoder they will be different
    # keylen is same as value len in decoder
    heads=[]
    for i in range(h):
        score=tf.matmul(wq[i](query),wk[i](value),transpose_b=True)
        score/=tf.math.sqrt(tf.dtypes.cast(keysize,tf.float32)) # batch x querylen x keylen
        alignment=tf.nn.softmax(score,axis=2)
        head=tf.matmul(alignment,wv[i](value))
        heads.append(head)
    
    heads=tf.concat(heads,axis=2) # adds up valuesize dims which here is 64 across 2 dim to get 128
    # batch x valuelen x 128
    heads=wo(heads)
    return heads
    

In [42]:
embedvar=tf.Variable(tf.random_normal(shape=[len(textwordind),modelsize]))


In [43]:
wq,wk,wv,wo,keysize=multiheadattention(modelsize,h)

In [44]:
embedvar

<tf.Variable 'Variable:0' shape=(25001, 128) dtype=float32_ref>

In [45]:
pes

<tf.Tensor 'Const:0' shape=(74, 128) dtype=float32>

In [46]:
sub1=[]
for i in range(x.shape[1]):
    embed=tf.nn.embedding_lookup(embedvar,tf.expand_dims(x[:,i],axis=1))
    #print(embed.get_shape())
    sub1.append(embed+pes[i,:])

subconcat1=tf.concat(sub1,axis=1)

In [47]:
#Since we have to add positional embeddings to embed variable thats why we are partioning x variable along sequence length
# and then adding positional encoding to embeddings

In [48]:
subconcat1

<tf.Tensor 'concat:0' shape=(?, 74, 128) dtype=float32>

In [49]:
subout=[]
for j in range(subconcat1.shape[1]):
    attention1=call(tf.expand_dims(subconcat1[:,j,:],axis=1),subconcat1,keysize,h,wq,wk,wv,wo)
    subout.append(attention1)


In [50]:
subout=tf.concat(subout,axis=1)

In [51]:
#One thing we can try may be here call(subconcat1,subconcat1.keysize,h,wq,wk,wv,wo)


In [52]:
subout=subout+subconcat1

In [53]:
subout=tf.keras.layers.BatchNormalization()(subout)
ffin=tf.keras.layers.Dense(4*modelsize,activation='relu')(subout)
ffin=tf.keras.layers.Dense(modelsize)(ffin)

In [54]:
ffout=ffin+subout
ffoutnorm=tf.keras.layers.BatchNormalization()(ffout)

In [55]:
wq2,wk2,wv2,wo2,keysize2=multiheadattention(modelsize,h)

In [56]:
subin2=ffoutnorm
subout2=[]
for j in range(subconcat1.shape[1]):
    attention2=call( tf.expand_dims(subin2[:,j,:],axis=1),subin2,keysize2,h,wq2,wk2,wv2,wo2)
    subout2.append(attention2)
subout2=tf.concat(subout2,axis=1)
    

In [57]:
subout2=subout2+subin2
subout2=tf.keras.layers.BatchNormalization()(subout2)
ffin2 = tf.keras.layers.Dense(4*modelsize,activation=tf.nn.relu)(subout2)
ffin2 = tf.keras.layers.Dense(modelsize)(ffin2)

In [58]:
ffout2=ffin2+subout2
ffoutnorm2=tf.keras.layers.BatchNormalization()(ffout2)

In [59]:
encoderoutput=ffoutnorm2

In [60]:
encoderoutput

<tf.Tensor 'batch_normalization_3/batchnorm/add_1:0' shape=(?, 74, 128) dtype=float32>

In [61]:
class decoder:
    def __init__(self,modelsize,h):
        
        self.modelsize=modelsize
        self.h=h
        self.dembedvar=tf.Variable(tf.random_normal(shape=(len(headwordind),self.modelsize)))
        self.dwq1,self.dwk1,self.dwv1,self.dwo1,self.dkeysize1=multiheadattention(self.modelsize,self.h)
        self.dwq2,self.dwk2,self.dwv2,self.dwo2,self.dkeysize2=multiheadattention(self.modelsize,self.h)
        self.damwq1,self.damwk1,self.damwv1,self.damwo1,self.damkeysize1=multiheadattention(self.modelsize,self.h)
        self.damwq2,self.damwk2,self.damwv2,self.damwo2,self.damkeysize2=multiheadattention(self.modelsize,self.h)
        self.batchnorm1=tf.keras.layers.BatchNormalization()
        self.midnorm=tf.keras.layers.BatchNormalization()
        self.dens1=tf.keras.layers.Dense(4*modelsize,activation=tf.nn.relu)
        self.dens2=tf.keras.layers.Dense(self.modelsize)
        self.batchnorm2=tf.keras.layers.BatchNormalization()
        self.dens3=tf.keras.layers.Dense(len(headwordind))
        self.batchnorm3=tf.keras.layers.BatchNormalization()
    def call(self,encoderoutput,yinput):
        deemb=[]
        for i in range(yinput.shape[1]):
            dmb=tf.nn.embedding_lookup(self.dembedvar,tf.expand_dims(yinput[:,i],axis=1))
            deemb.append(dmb)
    
    
        deemb=tf.concat(deemb,axis=1)
    
        botsubin=deemb
    
        botsubout1=[]
        for j in range(botsubin.shape[1]):
            values=botsubin[:,j,:]
            attention=call(tf.expand_dims(botsubin[:,j,:],axis=1),botsubin[:,:j,:],self.dkeysize1,self.h,self.dwq1,self.dwk1,self.dwv1,self.dwo1)
            botsubout1.append(attention)

        botsubout1=tf.concat(botsubout1,axis=1)
        botsubout1=botsubout1+botsubin
        botsubout1=self.batchnorm1(botsubout1)
    
    
        midsubin=botsubout1
    
        midsubout=[]
        for j in range(midsubin.shape[1]):
            datt=call(tf.expand_dims(midsubin[:,j,:],axis=1),encoderoutput,self.damkeysize1,self.h,self.damwq1,self.damwk1,self.damwv1,self.damwo1)
            midsubout.append(datt)
    
    
        midsubout1=tf.concat(midsubout,axis=1)
        midsubout11=midsubout1+midsubin
    
        midsubout12=self.batchnorm2(midsubout11)
    
        dffin=midsubout12
        dffout=self.dens1(dffin)
        dffout=self.dens2(dffout)
    
        dffout=dffout+dffin
        dffout=self.batchnorm3(dffout)
    
        logits1=self.dens3(dffout)
    
        return logits1    

In [62]:
#Important Realizations while creating Decoder 
# 1:- Create class of decoder  with all variables are defined in init function 
# 2:- It is very important to note that in call function of decoder there shouldnt be any new variable that is created as we need trained variables for prediction later on
# 3:- Input to decoder should be <start> token + remaining sentence 
# 4:- Output to decoder should be sentence +<end token>
# 5:- We training decoder in such a way that given previous word predict the next word of sequence 


In [63]:
decoder1=decoder(modelsize,h)

In [64]:
logits1=decoder1.call(encoderoutput,yinp)

In [65]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits1,labels=tf.one_hot(yout,len(headwordind))))

In [66]:
trainoptimizer=tf.train.AdamOptimizer(learning_rate=0.001)
trainstep=trainoptimizer.minimize(loss)

In [67]:
sess=tf.InteractiveSession()

In [68]:
sess.run(tf.global_variables_initializer())

In [69]:
batchsize=64

In [70]:
len(texttoken)

30000

In [71]:
count=0
trainloss=[]

In [72]:
for i in range(20):
    index=np.arange(30000)
    np.random.shuffle(index)
    for j in range(0,30000,100):
        ind=index[j:j+100]
        
        xinp=texttokenpad[ind]
        yinp1=headlineinpad[ind]
        yout1=headlineoutpad[ind]
    
        #print(xinp.shape)
        #print(yinp1.shape)
        #print(yout1.shape)
        _=sess.run(trainstep,feed_dict={x:xinp,yinp:yinp1,yout:yout1})
        loss1=loss.eval({x:xinp,yinp:yinp1,yout:yout1})
        count+=1
        trainloss.append(loss1)
        if count%20==0:
            print("Epoch:-",count," and loss is :-",loss1)

Epoch:- 20  and loss is :- 4.7343926
Epoch:- 40  and loss is :- 4.551646
Epoch:- 60  and loss is :- 4.3071337
Epoch:- 80  and loss is :- 4.4451585
Epoch:- 100  and loss is :- 4.3262453
Epoch:- 120  and loss is :- 4.2303386
Epoch:- 140  and loss is :- 4.224185
Epoch:- 160  and loss is :- 4.146503
Epoch:- 180  and loss is :- 4.127599
Epoch:- 200  and loss is :- 4.02826
Epoch:- 220  and loss is :- 4.010023
Epoch:- 240  and loss is :- 3.9344652
Epoch:- 260  and loss is :- 3.9576147
Epoch:- 280  and loss is :- 3.7862635
Epoch:- 300  and loss is :- 3.9367054
Epoch:- 320  and loss is :- 3.6643043
Epoch:- 340  and loss is :- 3.754556
Epoch:- 360  and loss is :- 3.5389647
Epoch:- 380  and loss is :- 3.5413046
Epoch:- 400  and loss is :- 3.4745688
Epoch:- 420  and loss is :- 3.4720492
Epoch:- 440  and loss is :- 3.5092423
Epoch:- 460  and loss is :- 3.5079372
Epoch:- 480  and loss is :- 3.4838758
Epoch:- 500  and loss is :- 3.3375401
Epoch:- 520  and loss is :- 3.5352547
Epoch:- 540  and loss is

Epoch:- 4280  and loss is :- 0.8561341
Epoch:- 4300  and loss is :- 0.952933
Epoch:- 4320  and loss is :- 0.96230227
Epoch:- 4340  and loss is :- 0.93125814
Epoch:- 4360  and loss is :- 0.91142726
Epoch:- 4380  and loss is :- 0.9744087
Epoch:- 4400  and loss is :- 0.977631
Epoch:- 4420  and loss is :- 1.0528629
Epoch:- 4440  and loss is :- 1.0132817
Epoch:- 4460  and loss is :- 1.0458969
Epoch:- 4480  and loss is :- 0.9271162
Epoch:- 4500  and loss is :- 1.086753
Epoch:- 4520  and loss is :- 0.82785463
Epoch:- 4540  and loss is :- 0.7640037
Epoch:- 4560  and loss is :- 0.7747284
Epoch:- 4580  and loss is :- 0.7491427
Epoch:- 4600  and loss is :- 0.83882153
Epoch:- 4620  and loss is :- 0.8327111
Epoch:- 4640  and loss is :- 0.7778619
Epoch:- 4660  and loss is :- 0.91110325
Epoch:- 4680  and loss is :- 0.8631939
Epoch:- 4700  and loss is :- 0.9310546
Epoch:- 4720  and loss is :- 0.9472607
Epoch:- 4740  and loss is :- 0.81522506
Epoch:- 4760  and loss is :- 1.0009971
Epoch:- 4780  and los

In [75]:
headindword=dict([(j,i) for i,j in headwordind.items()])

In [74]:
enout=encoderoutput.eval({x:texttokenpad[11].reshape(1,text)})
enout1=tf.constant(enout)

TypeError: 'Series' object cannot be interpreted as an integer

In [76]:
texttokenpad[0].shape

(74,)

In [77]:
def predict(word,enout1):
    deinput=tf.constant([[headwordind[word]]],dtype=tf.int64)
    prediction1=decoder1.call(enout1,deinput)
    firstval=prediction1.eval()
    word1=headindword[np.argmax(firstval)]
    return word1

In [78]:
textindword=dict([(j,i) for i,j in textwordind.items()])

In [79]:
def printsent(index):
    enout=encoderoutput.eval({x:texttokenpad[index].reshape(1,texttokenpad.shape[1])})
    enout1=tf.constant(enout)
    count=0
    word='<start>'
    li=[]
    while count<15:
        word=predict(word,enout1)
        li.append(word)
        count+=1
    print("predicted sentence:"," ".join(li))
    print("----")
    print("full text:-"," ".join([textindword[i] for i in texttokenpad[index]]))
    print("----")
    print("original summary:-"," ".join([headindword[i] for i in headlineoutpad[index]]))
    

In [81]:
printsent(23) # After training for 3000 stepsand 10 epochs

predicted sentence: jcb visit man <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
----
full text:- a man who has been operating jcb machine for decade karnataka <unk> kallakatta took his newlywed bride home in jcb machine decorated with balloons she was scared and refused to sit on it said kallakatta his wife agreed to sit on the machine when kallakatta assured he has been working with the machine every day for years <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
----
original summary:- jcb operator takes wife home in jcb machine after wedding <end> <pad> <pad> <pad> <pad> <pad> <pad>


In [ ]:
textindword[1]